## Towards Large Language Models Interacting with Knowledge Graphs Via Function Calling

Abstract:

This position paper introduces a new way for large language models (LLMs) to interact with Knowledge Graphs (KGs), especially for information extraction.
Most relation extraction approaches focus on finding the corresponding textual spans for subject, relation, and object in a text but ignore the actual state of the KG and the schema defined therein. In this work, the function-calling possibility of LLMs is explored to search for already defined entities in the KG and add information in the form of triples to the KG. A huge improvement over previous techniques is that it can also deal with various large ontologies / KGs and that the added information fits to a provided KG on a schema and instance level. Further validation checks can be added to increase the quality of the extracted statements. A small playground is provided to analyze the effectiveness of the approach.

In this notebook we provide a small playground to try thing out.

First setup everything:

In [ ]:
!git clone https://github.com/sven-h/LLMs4IE.git
!mv LLMs4IE/* .
!rm -r LLMs4IE

In [ ]:
!pip install -r requirements.txt

In [ ]:
# provide your OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = ""

In [1]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent


from KG_Create_Toolkit import KGCreateToolkit
from KG_Search_Toolkit import KGSearchToolkit


from rdf_graph import RdfGraph
model = RdfGraph()
#model = RdfGraph(source_file="myKB.ttl") 
#model = RdfGraph(query_endpoint="http://dbpedia.org/sparql")

base_uri = "http://myKB.org/"
return_full_uri = True
use_speaking_names = False
query_result_format = "csv"

tools = []
tools.extend(KGCreateToolkit(model=model, base_uri=base_uri, return_full_uri=return_full_uri, use_speaking_names=use_speaking_names).get_tools())
tools.extend(KGSearchToolkit(model=model, result_format=query_result_format).get_tools())

prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#import json
#from langchain_core.utils.function_calling import convert_to_openai_tool
#for tool in tools:
#    print(json.dumps(convert_to_openai_tool(tool), indent=2))
#    print("------------------------------------")

sentence = "The capital of France is Paris."

agent_executor.invoke({"input": (
    "Extract all possible information from sentences by searching for instances, properties, and classes."
    "In case no corresponding entity is found, create it with the corresponding functions. "
    "Finally, add a statement to the knowledge graph."
    "Before creating a property, search for it with function search_property. "
    "Extract all possible information from the following sentence: "
    f"'{sentence}'")})

kb = model.serialize_to_string()

c:\Users\shert\anaconda3\envs\myenv\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(




> Entering new AgentExecutor chain...

Invoking: `search_property` with `{'search_text': 'capital'}`


property_id,label,comment,domain_label,range_label

Invoking: `create_property` with `{'comment': 'The property that represents the capital of a country.', 'domain': 'Country', 'label': 'capital', 'range': 'City'}`


The domain of the property (which should be a class) does not exist in the graph. Search for it with function search_class. If it cannot be found, create it with create_class function and use the URI.
Invoking: `search_class` with `{'search_text': 'Country'}`


class_id,label,comment

Invoking: `create_class` with `{'comment': 'A class representing a country.', 'label': 'Country'}`


http://myKB.org/C3122976628735170500
Invoking: `create_property` with `{'comment': 'The property that represents the capital of a country.', 'domain': 'http://myKB.org/C3122976628735170500', 'label': 'capital', 'range': 'City'}`


The range of the property (which should be a class) does not

In [2]:
# finally print the resulting KG
print(kb)

@prefix ns1: <http://myKB.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ns1:C3122976628735170500 a rdfs:Class ;
    rdfs:label "Country" ;
    rdfs:comment "A class representing a country." .

ns1:C3436143458968470582 a rdfs:Class ;
    rdfs:label "City" ;
    rdfs:comment "A class representing a city." .

ns1:I3358560434636275274 a ns1:C3122976628735170500 ;
    rdfs:label "France" ;
    ns1:P8317186175532565255 ns1:I5123781352166096466 ;
    rdfs:comment "The country of France." .

ns1:P8317186175532565255 a rdf:Property ;
    rdfs:label "capital" ;
    rdfs:comment "The property that represents the capital of a country." ;
    rdfs:domain ns1:C3122976628735170500 ;
    rdfs:range ns1:C3436143458968470582 .

ns1:I5123781352166096466 a ns1:C3436143458968470582 ;
    rdfs:label "Paris" ;
    rdfs:comment "The capital city of France." .


